In [1]:
import numpy as np
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')

conn = sqlite3.connect('/Users/ograndberry/Desktop/berlin_startup_jobs_analysis/Database_Sqlite/db_berlin_jobs.db')
c = conn.cursor()

In [2]:
skills_final = pd.read_csv('final_skills.csv').drop('field1', axis=1)

In [3]:
skills_final['type'] = skills_final['type'].str.lower()

In [4]:
skills_final.dropna(inplace=True)

In [5]:
#skills_final['type'].replace('ux/ui', 'frontend')

In [6]:
def fixedtype(data):
    if data == 'scala':
        return 'bigdata'
    if data == 'machine learning':
        return 'bigdata'
    if data == 'jira':
        return 'project management tools'
    if data == 'grails':
        return 'groovy'
    if data == 'spark':
        return 'bigdata'
    if data == 'spring':
        return 'cloud'
    if data == 'objective-c':
        return 'iOS'
    if data == 'swift':
        return 'iOS'
    if data == 'hadoop':
        return 'bigdata'
    if data == 'google cloud platform':
        return 'cloud'
    if data == 'sass/less' :
        return 'CSS'
    if data == 'chef':
        return 'it automation'
    if data == 'apache frameworks':
        return 'cloud tools'
    if data == 'ux/ui':
        return 'frontend'
    if data == 'redux':
        return 'javaScript tools'
    if data == 'rabbitmq':
        return 'messaging protocol'
    if data == 'gulp':
        return 'it automation'   
    if data == 'scikit-learn / deep learning':
        return 'bigdata'
    if data == 'jquery':
        return 'javascript' 
    if data == 'bigdata':
        return 'bigdata'
    if data == 'continuous deployment':
        return 'continuous integration'
    if data == 'git':
        return 'repository management services'
    else:
        return data

#change big clusters

#'Big Data'#'Scala'
#'Big Data'#'Machine Learning'
#'Project Management Tools'#'Jira'
#'Big Data' #spark
#'Cloud'#'Spring'
#'iOS'#'Objective-C'
#'iOS'#'Swift'
#'Big Data' #hadoop
#'Sass/LESS' #sass
#'IT Automation'#'Chef'
#'Cloud Tools'#'Apache Frameworks'
#'Big Data' #spark
#'Frontend'#'UI/UX'
#'JavaScript Tools'#'Redux'
#'Messaging Protocol'#'RabbitMQ'
#'IT Automation'#'Gulp'
#'scikit-learn / deep learning'

#'Spark' #apache-spark

In [7]:
#skills_final.loc[:, 'type'] = skills_final.apply(fixedtype, axis = 1)

skills_final['type2'] = skills_final['type'].apply(fixedtype)

In [8]:
skills_final[skills_final['type']=='jquery']

,job_id,skill,type,type2
3613,611,jquery,jquery,javascript
3614,617,jquery,jquery,javascript
3615,630,jquery,jquery,javascript
3616,650,jquery,jquery,javascript
3617,653,jquery,jquery,javascript
3618,684,jquery,jquery,javascript
3619,693,jquery,jquery,javascript
3620,699,jquery,jquery,javascript
3621,701,jquery,jquery,javascript
3622,17,JQuery,jquery,javascript


# Manual clustering

In [9]:
#grab the rows from the skills_final table tagged as 'mixed'
skills_mixed = skills_final[skills_final['type'] == 'mixed']
skills_mixed.dropna(inplace=True)
skills_mixed.head()

,job_id,skill,type,type2
2,556,ceph,mixed,mixed
4,563,engineer,mixed,mixed
7,567,opennebula,mixed,mixed
11,572,zeromq,mixed,mixed
14,574,dijkstra,mixed,mixed


In [10]:
skills_mixed['skill'] = skills_mixed['skill'].str.lower()
skills_mixed.drop('type', axis=1,inplace=True)
skills_mixed.columns = ['job_id','skill','type']

In [11]:
skills_mixed.head()

,job_id,skill,type
2,556,ceph,mixed
4,563,engineer,mixed
7,567,opennebula,mixed
11,572,zeromq,mixed
14,574,dijkstra,mixed


In [12]:
#import the skills3.csv holding the manual clusters made using a spreadsheet
skills_rawdata = pd.read_csv('DB/skills_t.csv')[['type','skill']]
skills_rawdata.columns = ['group','skill']
skills_rawdata.dropna(inplace=True)

In [13]:
skills_mixedi = pd.merge(skills_mixed, skills_rawdata , how='left', on='skill')
skills_mixedi.drop(['type'],axis=1, inplace= True)

In [14]:
skills_mixedi.columns = ['job_id','skill','type']
skills_mixedi.head()

,job_id,skill,type
0,556,ceph,bigdata
1,563,engineer,mixed
2,567,opennebula,cloud
3,572,zeromq,zeromq
4,574,dijkstra,bigdata


In [15]:
skills_final.head()
skills_final.drop(['type'],axis=1, inplace= True)
skills_final.columns = ['job_id','skill','type']


In [16]:
skills_final.head()

,job_id,skill,type
0,553,r,r
1,554,r,r
2,556,ceph,mixed
3,557,qa,qa
4,563,engineer,mixed


In [17]:
#creation of a new dataframe excluding the "mixed' types and replacing them with the 'skills_mixed' df.
skills_final2 = skills_final[skills_final['type'] != 'mixed']
final_skills_table = pd.concat([skills_final2, skills_mixedi])


In [18]:
#what's left as mixed
len(final_skills_table[final_skills_table['type'] == 'mixed'])

396

In [19]:
final_skills_table['type'].unique()

array(['r', 'qa', 'backend', 'bigdata', 'continuous integration', 'c/c++',
       'aws', 'c#', '.net', 'microservices', 'frontend',
       'open source olap', 'python-based', 'data viz tools', 'cloud tools',
       'algorithm', 'terraform', 'javascript tools', 'continuous delivery',
       'wordpress', 'testing', 'hibernate', 'sql',
       'repository management services', 'cloud', 'php frameworks',
       'node.js', 'kotlin', 'jvm', 'ruby', 'groovy', 'angular.js',
       'erlang', 'atlassian', 'ios', 'nosql', 'memcached', 'js compiler',
       'scala frameworks', 'yarn', 'elastic stack', 'perl', 'php',
       'bdd/tdd', 'restful apis', 'sysadmin', 'e-commerce platform', 'css',
       'messaging protocol', 'iOS', 'react.js', 'http', 'python', 'django',
       'agile', 'unix/linux', 'containerization platforms', 'java',
       'javascript', 'devops', 'bash', 'go', 'html', 'azure', 'CSS',
       'redis', 'api(s)', 'google packages', 'it automation', 'mvc',
       'shell', 'design', 'java

In [22]:
final_skills_table = final_skills_table.set_index('job_id')

In [23]:
final_skills_table.to_sql('Skills', conn, if_exists='replace')